In [55]:
import pandas as pd
import numpy as np
import scipy as sp
import matplotlib as plt

In [ ]:
#Load csv of cognitive data for each subject
cog_scores = pd.read_csv(open('/media/corey/4TB-WDBlue/data-thesis/cognitive_data_251.csv'),skiprows=1,delimiter=';')
#print(cog_scores.loc[3,]) #Print statement to check if the data were properly imported

In [46]:
#Sort subjects into groups [Control(0) , Non-affective(2), Affective(1)]
controls = cog_scores.loc[cog_scores['phenotype_description'] == 0]
affective = cog_scores.loc[cog_scores['phenotype_description'] == 1]
n_affective = cog_scores.loc[cog_scores['phenotype_description'] == 2]
#Print number of subjects in each group
print(len(controls))
print(len(affective))
print(len(n_affective))

68
57
126


In [67]:
controls.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 68 entries, 3 to 233
Data columns (total 50 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   src_subject_id                         68 non-null     int64  
 1   phenotype                              68 non-null     int64  
 2   phenotype_description                  68 non-null     int64  
 3   sex                                    68 non-null     int64  
 4   interview_age                          68 non-null     float64
 5   sestot                                 67 non-null     float64
 6   mot_edscale                            66 non-null     float64
 7   acpt01_auditory_t1                     66 non-null     float64
 8   acpt01_auditory_t3                     66 non-null     float64
 9   acpt01_auditory_t4                     66 non-null     float64
 10  acpt01_auditory_t5                     66 non-null     float64
 11  acpt01_

In [ ]:
#Calculate descriptive statistics of cognitive scores for each group


In [ ]:
#Plot histograms

In [120]:
#Add column specifically for Non-affective psychosis (i.e., group dummy var)
cog_no_na = cog_scores.dropna()
non_aff = np.zeros(len(cog_no_na))
for n in range (0,len(cog_no_na)-1):
    if cog_no_na.iloc[n,2] == 2:
        non_aff[n] = 1
#cog_no_na.loc[cog_no_na["phenotype_description"]==2,"phenotype_description"]]=0
cog_no_na["phenotype_description"] = np.where(cog_no_na['phenotype_description']==2,0,cog_no_na['phenotype_description'])
cog_no_na['non_aff'] = non_aff
print(cog_no_na.iloc[:,])

     src_subject_id  phenotype  phenotype_description  sex  interview_age  \
0              1048          1                      0    0          290.0   
1              1050          1                      0    1          347.0   
2              1051          1                      0    0          219.0   
3              1052          0                      0    0          249.0   
4              1053          1                      0    0          290.0   
..              ...        ...                    ...  ...            ...   
246            1041          1                      0    0          225.0   
247            1043          1                      0    0          283.0   
248            1044          1                      0    0          269.0   
249            1045          1                      0    0          284.0   
250            1047          1                      0    0          264.0   

     sestot  mot_edscale  acpt01_auditory_t1  acpt01_auditory_t3  \
0      

/tmp/ipykernel_6722/798920553.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cog_no_na["phenotype_description"] = np.where(cog_no_na['phenotype_description']==2,0,cog_no_na['phenotype_description'])
/tmp/ipykernel_6722/798920553.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cog_no_na['non_aff'] = non_aff


In [74]:
#Test for difference in means (with ANOVA, control for sex and age)
import statsmodels.api as sm
from statsmodels.formula.api import ols
model = ols("acpt01_auditory_t4 ~ C(phenotype_description)",data=cog_scores).fit()
aov_table = sm.stats.anova_lm(model, typ=2)
aov_table

,sum_sq,df,F,PR(>F)
C(phenotype_description),6881.789759,2.0,15.94494,3.149081e-07
Residual,51791.650799,240.0,NaN,NaN


In [77]:
#Use scikit learn for multiple regression
from sklearn.linear_model import LinearRegression

mod = LinearRegression()
mod.fit(cog_no_na[['sex','interview_age']],cog_no_na['acpt01_auditory_t4'])
print(mod.intercept_)
print(mod.coef_)


(148, 50)
75.06928770144417
[-1.81574845  0.02299814]


In [122]:
import statsmodels as ssm
x = cog_no_na[['sex','interview_age','phenotype_description','non_aff']]
y = cog_no_na[['acpt01_auditory_t4']]

X = ssm.tools.tools.add_constant(x)
model = ssm.regression.linear_model.OLS(y,X).fit()
pred = model.summary()
pred

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:     acpt01_auditory_t4   R-squared:                       0.144
Model:                            OLS   Adj. R-squared:                  0.120
Method:                 Least Squares   F-statistic:                     5.991
Date:                Sun, 30 Apr 2023   Prob (F-statistic):           0.000174
Time:                        12:00:00   Log-Likelihood:                -609.16
No. Observations:                 148   AIC:                             1228.
Df Residuals:                     143   BIC:                             1243.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
=========================================================================================
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                    95.2811      8.875     10.736      0.000      77.738     112.824
sex                      -2.3003      2.652     -0.867      0.387      -7.543       2.942
interview_age            -0.0189      0.029     -0.652      0.516      -0.076       0.038
phenotype_description    -6.7514      3.910     -1.727      0.086     -14.481       0.978
non_aff                 -13.9263      2.924     -4.762      0.000     -19.707      -8.146
==============================================================================
Omnibus:                       37.810   Durbin-Watson:                   1.654
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               71.757
Skew:                          -1.163   Prob(JB):                     2.62e-16
Kurtosis:                       5.496   Cond. No.                     2.06e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.06e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [125]:
x = cog_no_na[['sex','interview_age','phenotype_description','non_aff']]
y = cog_no_na[['acpt01_auditory_t5']]

X = ssm.tools.tools.add_constant(x)
model = ssm.regression.linear_model.OLS(y,X).fit()
pred = model.summary()
pred

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:     acpt01_auditory_t5   R-squared:                       0.113
Model:                            OLS   Adj. R-squared:                  0.088
Method:                 Least Squares   F-statistic:                     4.536
Date:                Mon, 01 May 2023   Prob (F-statistic):            0.00177
Time:                        19:15:15   Log-Likelihood:                -915.89
No. Observations:                 148   AIC:                             1842.
Df Residuals:                     143   BIC:                             1857.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
=========================================================================================
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                   538.5187     70.511      7.637      0.000     399.140     677.898
sex                      49.3233     21.070      2.341      0.021       7.674      90.972
interview_age             0.1464      0.230      0.637      0.525      -0.308       0.601
phenotype_description   -42.1024     31.066     -1.355      0.177    -103.511      19.306
non_aff                  64.3137     23.233      2.768      0.006      18.390     110.238
==============================================================================
Omnibus:                       30.658   Durbin-Watson:                   2.336
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               56.165
Skew:                           0.959   Prob(JB):                     6.37e-13
Kurtosis:                       5.331   Cond. No.                     2.06e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.06e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [ ]:
#Iterate regression over each cognitive score
cog_no_na_ofint = cog_no_na.drop(columns=["src_subject_id","phenotype","sestot","mot_edscale"])
models = []
preds = []
indep_list = ['sex','interview_age','phenotype_description','non_aff']
indep = cog_no_na_ofint[['sex','interview_age','phenotype_description','non_aff']]
X = ssm.tools.tools.add_constant(indep)
for x, c in enumerate(cog_no_na_ofint):
    if c in indep:
        continue
    dep = cog_no_na_ofint[[c]]
    model = ssm.regression.linear_model.OLS(dep,X).fit()
    pred = model.summary()
    models.append(models)
    preds.append(pred)
    
print(len(models))
print(len(preds))
print(cog_no_na_ofint.shape)
print(preds[0])


In [163]:
cog_no_na_ofint.columns.get_loc('phenotype_description')
print(cog_no_na_ofint.shape[1])

47


In [195]:
#Return ols prediction summary of a given dependent variable 
#(takes dependent variable of interest, data frame, and structure containing summaries as inputs)

#Note: If using the entire dataframe as input,
#this function assumes that any dependent variables are stored in a continuous set of indices:
#e.g., [3,4,5,6,...45,46,47] and NOT [1,8,15,...,22,25,33...]

#If a dataframe without the columns containing indpendent variables is used as an input,
#this inconsistency is not an issue 
def return_summary(dep_var, df, summaries): 
     shift = df.shape[1] - len(summaries) - 1
     idx = df.columns.get_loc(dep_var)
     if df.shape[1] == len(summaries):
          return summaries[idx]
     else:
          return summaries[idx-shift]

In [196]:
return_summary('acpt01_auditory_t5', cog_no_na_ofint, preds)

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:     acpt01_auditory_t5   R-squared:                       0.113
Model:                            OLS   Adj. R-squared:                  0.088
Method:                 Least Squares   F-statistic:                     4.536
Date:                Mon, 01 May 2023   Prob (F-statistic):            0.00177
Time:                        19:29:02   Log-Likelihood:                -915.89
No. Observations:                 148   AIC:                             1842.
Df Residuals:                     143   BIC:                             1857.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
=========================================================================================
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                   538.5187     70.511      7.637      0.000     399.140     677.898
sex                      49.3233     21.070      2.341      0.021       7.674      90.972
interview_age             0.1464      0.230      0.637      0.525      -0.308       0.601
phenotype_description   -42.1024     31.066     -1.355      0.177    -103.511      19.306
non_aff                  64.3137     23.233      2.768      0.006      18.390     110.238
==============================================================================
Omnibus:                       30.658   Durbin-Watson:                   2.336
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               56.165
Skew:                           0.959   Prob(JB):                     6.37e-13
Kurtosis:                       5.331   Cond. No.                     2.06e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.06e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [188]:
#Drop independent variables from df
cog_dep_only = cog_no_na_ofint.drop(columns=indep_list)
print(cog_dep_only.shape)

(148, 43)


In [219]:
return_summary('acpt01_auditory_t9',cog_dep_only,preds)

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:     acpt01_auditory_t9   R-squared:                       0.073
Model:                            OLS   Adj. R-squared:                  0.047
Method:                 Least Squares   F-statistic:                     2.831
Date:                Mon, 01 May 2023   Prob (F-statistic):             0.0268
Time:                        19:29:02   Log-Likelihood:                -583.77
No. Observations:                 148   AIC:                             1178.
Df Residuals:                     143   BIC:                             1193.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
=========================================================================================
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                    99.3864      7.476     13.293      0.000      84.608     114.165
sex                      -0.5638      2.234     -0.252      0.801      -4.980       3.852
interview_age            -0.0058      0.024     -0.237      0.813      -0.054       0.042
phenotype_description    -1.0848      3.294     -0.329      0.742      -7.596       5.426
non_aff                  -7.5667      2.463     -3.072      0.003     -12.436      -2.697
==============================================================================
Omnibus:                      138.238   Durbin-Watson:                   1.757
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1847.626
Skew:                          -3.440   Prob(JB):                         0.00
Kurtosis:                      18.883   Cond. No.                     2.06e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.06e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [220]:
#Print full list of summaries (open in text editor for all)
print(preds)

[<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
Dep. Variable:     acpt01_auditory_t1   R-squared:                       0.142
Model:                            OLS   Adj. R-squared:                  0.118
Method:                 Least Squares   F-statistic:                     5.930
Date:                Mon, 01 May 2023   Prob (F-statistic):           0.000192
Time:                        19:29:02   Log-Likelihood:                -397.61
No. Observations:                 148   AIC:                             805.2
Df Residuals:                     143   BIC:                             820.2
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------

In [ ]:
#Include interaction term(s) and re-run

In [ ]:
#Plot results

#Check residuals for heteroscedasticity

#